In [10]:
from dotenv import load_dotenv
from dataclasses import dataclass
from langchain.agents import AgentState, create_agent
from langchain.tools import tool, ToolRuntime
from langgraph.types import Command
from langchain.messages import ToolMessage
from langchain.agents.middleware import wrap_model_call, dynamic_prompt, HumanInTheLoopMiddleware
from langchain.agents.middleware import ModelRequest, ModelResponse
from typing import Callable
from langchain_openai import ChatOpenAI
from langgraph.checkpoint.memory import InMemorySaver
import os

load_dotenv()


OPENROUTER_API_KEY = os.getenv("OPENROUTER_API_KEY")
if not OPENROUTER_API_KEY:
    raise EnvironmentError("Установите OPENROUTER_API_KEY в файле .env")


llm = ChatOpenAI(
    model="google/gemini-3-flash-preview",
    base_url="https://openrouter.ai/api/v1",
    api_key=OPENROUTER_API_KEY,
    temperature=0.0
)

In [11]:
@dataclass
class EmailContext:
    email_adress: str = "julie@example.com"
    password: str = "password123"

In [12]:
class AuthenticatedState(AgentState):
    authenticated: bool

In [13]:
@tool
def check_inbox() -> str:
    """Check the inbox for recent email"""
    return """
    Привет, Джули!
    Я буду в городе на следующей неделе и хотела бы пригласить тебя выпить кофе. 
    С наилучшими пожеланиями, Джейн(jane@example.com)
    """

@tool
def send_email(to: str, subject: str, body: str) -> str:
    """Send an response email"""
    return f"Письмо отправлено {to} с темой {subject} и текстом {body} "

@tool
def authenticate(email: str, password: str, runtime: ToolRuntime) -> Command:
    """Authenticate the user with the given email and password"""
    if email == runtime.context.email_adress and password == runtime.context.password:
        return Command(update={
            "authenticated": True,
            "messages": [ToolMessage(
                "Successfully authenticated",
                tool_call_id=runtime.tool_call_id
            )]
        })
    else:
        return Command(update={
            "authenticated": False,
            "messages": [ToolMessage(
                "Authentication failed",
                tool_call_id=runtime.tool_call_id
            )]
        })

In [14]:
@wrap_model_call
def dynamic_tool_call(request: ModelRequest, 
                      handler: Callable[[ModelRequest], ModelResponse]) -> ModelResponse:
    """Allow read inbox and send email tools only if user provides correct email and password"""

    authenticated = request.state.get("authenticated")

    if authenticated:
        tools = [check_inbox, send_email]
    else:
        tools = [authenticate]
    
    request = request.override(tools=tools)
    return handler(request)

In [15]:
authenticated_prompt = """
You are a helpful email assistant.
CRITICAL RULES:
1. NEVER describe actions — ALWAYS perform them using tools.
2. To send an email, you MUST call the `send_email` tool.
3. To check inbox, you MUST call the `check_inbox` tool.
4. If user asks to send/reply/write email — use `send_email` tool immediately.
"""

unauthenticated_prompt = "You can only authenticate users. Ask for email and password."

@dynamic_prompt
def dynamic_prompt(request: ModelRequest) -> str:
    authenticated = request.state.get("authenticated")
    return authenticated_prompt if authenticated else unauthenticated_prompt

In [16]:
agent = create_agent(
    model=llm,
    tools=[authenticate, check_inbox, send_email],
    checkpointer=InMemorySaver(),
    context_schema=EmailContext,
    middleware=[
        dynamic_tool_call,
        dynamic_prompt,
        HumanInTheLoopMiddleware(
            interrupt_on={
                "authenticate": False,
                "check_inbox": False,
                "send_email": True
            }
        )
    ]
)

In [17]:
from langchain.messages import HumanMessage


config = {"configurable": {"thread_id": "1"}}

# 1. Аутентификация (у тебя уже работает!)
auth_response = agent.invoke(
    {"messages": [HumanMessage(
        content="My email is julie@example.com and my password is password123"
    )]},
    context=EmailContext(),
    config=config
)
print("Auth result:", auth_response["messages"][-1].content)

# 2. Чёткая команда на отправку письма
response = agent.invoke(
    {"messages": [HumanMessage(
        content="Ответь на письмо от Jane, что я согласен встретиться на кофе."
    )]},
    context=EmailContext(),
    config=config
)

# 3. Теперь должно быть прерывание!
if "__interrupt__" in response:
    print("📄 Черновик письма:")
    print(response["__interrupt__"][0].value["action_requests"][0]["args"]["body"])
    
    # 4. Подтверждаем отправку
    final_response = agent.invoke(
        Command(resume={"decisions": [{"type": "approve"}]}),
        config=config
    )
    print("\n✅ Результат:", final_response["messages"][-1].content)
else:
    print("❗ Нет прерывания. Агент ответил:")
    print(response["messages"][-1].content)

c:\Users\ryash\AppData\Local\Programs\Python\Python312\Lib\site-packages\pydantic\main.py:463: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected `none` - serialized value may not be as expected [input_value=EmailContext(email_adress... password='password123'), input_type=EmailContext])
  return self.__pydantic_serializer__.to_python(


Auth result: You have been successfully authenticated. How can I help you today?
❗ Нет прерывания. Агент ответил:
Я готов помочь. Чтобы я мог отправить ответ, мне нужно найти письмо от Jane.

Подскажите, пожалуйста:
1. У вас есть доступ к инструментам для работы с почтой (например, поиск писем)?
2. Или мне просто составить текст ответа, который вы отправите сами?

Если вы хотите, чтобы я составил текст, вот вариант:

**Тема:** Re: Встреча на кофе

Привет, Джейн!
Я согласен встретиться на кофе. Давай обсудим детали. Когда тебе будет удобно?

С уважением,
Джули


In [18]:
print(response["messages"][-1].tool_calls)

[]
